In [26]:
import urllib.request
import datetime
import time
import json
from config import *
import math
import requests
from bs4 import BeautifulSoup
import folium

In [27]:
def get_request_url(url):
    
    req = urllib.request.Request(url)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [28]:
#[CODE 1]

def SanggaInfoService(pageNo,numOfRows,sigunguCd,bjdongCd):
    
    end_point = "http://api.suwon.go.kr/openapi-data/service/SanggaInfo/getHjdsearchInfo"
    
    parameters = "?pageNo=" + str(pageNo)
    parameters += "&numOfRows=" + str(numOfRows)
    parameters += "&sigunguCd=" + str(sigunguCd)
    parameters += "&bjdongCd=" + str(bjdongCd)
    parameters += "&ServiceKey=" + "TIyuV2FHFQSS46wIlsJNrwYGs9SqQEDOtNb0XZjVWp8k6pVUitI9BsXYzNGEYhUmLuovIzoeoIEA3fbXwqQwTg%3D%3D"
    
    global url
    
    url = end_point + parameters
    
    global retData
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return 

In [29]:
sigunguCd_suwon = [41111,41113,41115,41117]
sigungu_suwon = ['장안구','권선구','팔달구','영통구']

In [16]:
ratio = []

for i in sigunguCd_suwon :
    
    SanggaInfoService(1,5000,i,00000)

    soup = BeautifulSoup(retData, "html.parser")
    items = soup.find_all("item")

    food = []
    cafe = []
    
    for i in items :
        if str(i.indslclsnm.text) == "음식" :
            food.append(i)

    for p in food :
        if str(p.indsmclsnm.text) == "커피점/카페" :
            cafe.append(p)

    ratio_ind = len(cafe)/len(food)
    ratio.append(ratio_ind)

ratio

[0.06949300699300699,
 0.06954436450839328,
 0.1081282624906786,
 0.08934953538241601]

In [17]:
geo_path = '수원시_geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [18]:
map = folium.Map([37.292, 127.028], zoom_start = 12)
map.choropleth(geo_data = geo_str)
map

In [22]:
import pandas as pd

In [23]:
# 상대적 표준화

s = pd.Series(ratio,
              index=sigungu_suwon)
s = s/s.max()
s

장안구    0.642690
권선구    0.643165
팔달구    1.000000
영통구    0.826329
dtype: float64

In [25]:
map = folium.Map([37.292, 127.028], zoom_start = 12)
map.choropleth(geo_data = geo_str, data = s,
               columns = [s.index, s],
               key_on = 'feature.id',
               fill_color = 'PuRd',
               legend_name = '구별 음식점 내 카페 비율')
map

In [31]:
total = []

for i in sigunguCd_suwon :
    
    SanggaInfoService(1,5000,i,00000)

    soup = BeautifulSoup(retData, "html.parser")
    items = soup.find_all("item")

    food_2 = []
    cafe_2 = []
    
    for i in items :
        if str(i.indslclsnm.text) == "음식" :
            food_2.append(i)

    for p in food_2 :
        if str(p.indsmclsnm.text) == "커피점/카페" :
            cafe_2.append(p)
    
    total.append(len(cafe_2))
    
total

[159, 87, 145, 125]

In [36]:
cafe_series = pd.Series(total,
              index=sigungu_suwon)
cafe_series

장안구    159
권선구     87
팔달구    145
영통구    125
dtype: int64

In [39]:
map_cafe = folium.Map([37.292, 127.028], zoom_start = 12)
map_cafe.choropleth(geo_data = geo_str, data = cafe_series,
               columns = [cafe_series.index, cafe_series],
               key_on = 'feature.id',
               fill_color = 'PuRd',
               legend_name = '구별 카페 갯수')
map_cafe